In [1]:
%matplotlib inline 

import numpy as np
import pandas as pd
from scipy.stats import pearsonr 
import matplotlib.pyplot as plt
import glob
import os
import time
import xarray as xr

# import dask.array as da
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster

import matplotlib.patheffects as PathEffects

### load lidar xarray

In [4]:
fname = '~/Documents/projects/thesis/results/output/compiled_SUPERsnow.nc' #ARS
# fname = '~/Documents/research/sierra/data/compiled_SUPERsnow.nc' #BSU
#~~~~ ds full
ds = xr.open_dataset(fname,  chunks={'time':1,'x':1000,'y':1000})
ds.close()

#~~~~ ds peak
dpeak = ds.isel(time=[0,7,18,30,42,49]) #0,7,22,28,41,49]
dpeak.close()

ds = dpeak
ds.close()

### load terrain xarray

In [13]:
fpath = glob.glob('/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/equal_extent/*.nc')
fpath


['/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/equal_extent/hillshade.nc',
 '/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/equal_extent/roughness.nc',
 '/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/equal_extent/slope.nc',
 '/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/equal_extent/surface_classification.nc',
 '/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/equal_extent/aspect.nc']

In [2]:
# ds_terr=xr.open_mfdataset(fpath, concat_dim='space') #, chunks={'x':1000, 'y':1000}, parallel=True) # .rename({'Band1':'snow'}).drop('transverse_mercator') combine='nested',
# ds_terr

# hil = xr.open_dataset(fpath[0])
# print(hil)

# asp = xr.open_dataset(fpath[4])
# print(asp)

# slp = xr.open_dataset(fpath[2])
# print(slp)

# rou = xr.open_dataset(fpath[1])
# print(rou)

# sur = xr.open_dataset(fpath[3])
# print(sur)

### flatten

In [ ]:
#terrain
dem_flat = ds.dem.values.flatten()
aspect_flat = ds_aspect.values.flatten() #need to rename from band1 likely
slope_flat = ds_slope.values.flatten()

In [ ]:
# data
yrly_means = ds.snow.groupby('time.year').mean('time')
yrly_means = yrly_means.values.flatten()

yrly_std = ds.snow.groupby('time.year').std('time')
yrly_std = yrly_means.values.flatten()


### plot

In [ ]:

fig = plt.figure(figsize=(15, 10))
plt.hexbin(dem_flat, yrly_means, mincnt=1)
plt.xlabel('Elevation [m]')
plt.ylabel('$\sigma$')
plt.title('Snow depth [50m] STD by elevation')
plt.colorbar()
plt.show()
plt.savefig('figs/elevation_density_13-16', dpi=600)